In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.0


In [4]:
# Instalação de todas as bibliotecas necessárias.
# P.S: o PYP, no windows, deve estar atualizado, caso não esteja, seguir os passos que aparecerão na mensagem de WARNING
!pip install -q beautifulsoup4
!pip install -q pandas
!pip install -q tabulate
!pip install -q urllib3
!pip install -q requests
!pip install -q lxml
!pip install -q html5lib

You should consider upgrading via the 'c:\users\ágara andermann\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\ágara andermann\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
# Bibliotecas utilizadas
import requests, bs4, urllib.request, json, pandas as pd
from tabulate import tabulate

In [2]:
# Função que retorna o conteúdo da URL
def make_soup(url):
    r = requests.get(url)
    soup = bs4.BeautifulSoup(r.text, 'lxml')
    return soup

# Função que retorna o FORM onde estará o botão com a requisição
def get_form(soup):
    form = soup.form
    return form

# Função que retorna a URL absoluta a partir da action do form da página inicial
def get_action(form, base_url):
    action = form['action']
    action_modified = action.split("/")[2]
    #action is reletive url, convert it to absolute url
    abs_action = urllib.parse.urljoin(base_url, action_modified)
    return abs_action

# Função que retorna os parâmetros para serem enviados no POST da página inicial
# O único que é required é o UF
def get_form_data(form, UF):
    data = {}   
    for inpInput, inpSelect in zip(form('input'), form('select')):
        data[inpSelect['name']] = UF
        if inpInput.value == None:
            data[inpInput['name']] = ' '
        else:
            data[inpInput['name']] = inpInput.value 
            data['Bairro'] = ''    
    return data

# Função que retorna os parâmetros para serem enviados no POST nas páginas de resultado (no botão SEGUINTE)
# Aqui, preciso do UF e da página inicial e final, contidas no form do resultado
def get_form_data_proxima(UF, pgIni, pgFim):
    dataProxima = {}     
    dataProxima['UF'] = UF
    dataProxima['Bairro'] = ''    
    dataProxima['pagini'] = pgIni
    dataProxima['pagfim'] = pgFim
    return dataProxima

# Função que retorna a URL absoluta a partir da action do form da página de resultado
def get_action_proxima(formProxima, base_url):
    action_modified_proxima = formProxima.split("/")[2]
    #action is reletive url, convert it to absolute url
    abs_action_proxima = urllib.parse.urljoin(base_url, action_modified_proxima)
    return abs_action_proxima

In [3]:
while True:
    try:
        if __name__ == '__main__':
                        
            url = 'http://www.buscacep.correios.com.br/sistemas/buscacep/buscaFaixaCep.cfm'
            
            # Digito a sigla do estado que quero fazer a raspagem de dados
            UF_escolha = str(input('Digite a sigla de algum estado: '))
            
            # Lista com as siglas corretas dos estados 
            listaEstados = ['AC','AL','AM','AP','BA','CE','DF','ES','GO','MA','MG','MS',\
                            'MT','PA','PB','PE','PI','PR','RJ','RN','RO','RR','RS','SC','SE','SP','TO']
            
            # Verifico a sigla digitada com a sigla contida na lista
            if UF_escolha.upper() not in listaEstados:
                print("Estado não existente, digite novamente.")
                UF_escolha = None
            else:
                UF_escolha = UF_escolha.upper()
            
            # DIV que receberá todas as tabelas no final
            divTables = bs4.BeautifulSoup('<div></div>')
            
            # Como não consigo pegar na página de resultado inicial o valor da pagIni e pagFim (para acessar as outras páginas),
            # esse LOOP (a partir do 1) vai percorrendo cada página com os parâmetros necessários pagIni e pagFim, 
            # que são separados por 49 linhas.
            # ex: pagIni=1 e pagFim=50 (segunda página dos resultados), pagIni=51 e pagFim=100 (terceira página dos resultados)
            for i in range(0,20):
                pagI = 1+(50*i)
                pagF = 50+(50*i)
                if i==0:
                    soup = make_soup(url)
                    form = get_form(soup)
                    action = get_action(form, url)
                    data = get_form_data(form, UF_escolha)
                    # Realizo o POST na página inicial que me traz os resultados (separados por páginas)
                    rResult = requests.post(action, data=data)
                    soupResult = bs4.BeautifulSoup(rResult.content, "html5lib")
                    # Tabela contendo o resultado
                    tableInit = soupResult.find_all("table", {"class":"tmptabela"})[1]
                    # Guardo o resultado na DIV
                    divTables.append(tableInit)              
                else:
                    form_proxima = soupResult.find('form')
                    form_action_proxima = form_proxima['action']
                    dataNew = get_form_data_proxima(UF_escolha, pagI, pagF)
                    action_proxima = get_action_proxima(form_action_proxima, url)
                    # Realizo o POST nas páginas de resultados para acessar a página seguinte
                    rResult_proxima = requests.post(action_proxima, data=dataNew)
                    soupResult = bs4.BeautifulSoup(rResult_proxima.content, "html5lib")
                    # Tabela contendo o resultado
                    tableInit = soupResult.find_all("table", {"class":"tmptabela"})[0]
                    # Guardo o resultado na DIV
                    divTables.append(tableInit)
               
    except TypeError:
        # Esse except indica que ele só tinha uma página de resultado
        print("\nRaspagem das faixas de CEP do estado "+ UF_escolha +" concluída.")
        break
    except IndexError:
        # Esse except indica que ele chegou na última página de resultado
        print("\nRaspagem das faixas de CEP do estado "+ UF_escolha +" concluída.")
        break    
    else:
        break

Digite a sigla de algum estado: sc

Raspagem das faixas de CEP do estado SC concluída.


In [4]:
# Caso seja necessário mostrar todas as linas do resultado:
# pd.set_option('display.max_rows', None)

# Quantidade de linhas
lenTables = len(divTables.find_all('table'))

# Crio o DF
df = pd.concat([pd.concat(pd.read_html(str(divTables.find_all('table')[i])), axis=0) for i in range(0,lenTables)], ignore_index=True)

# Retiro as LOCALIDADES duplicadas
df_noDups = df.drop_duplicates(subset=['Localidade'])

# Crio o INDEX (iniciando por 1)
df_noDups.index = df_noDups.index + 1

# Printo o resultado no formato tabela
df_noDups

,Localidade,Faixa de CEP,Situação,Tipo de Faixa
1,Abdon Batista,89636-000 a 89637-999,Não codificada por logradouros,Total do município
2,Abelardo Luz,89830-000 a 89831-999,Não codificada por logradouros,Total do município
3,Agrolândia,88420-000 a 88429-999,Não codificada por logradouros,Total do município
4,Agronômica,89188-000 a 89189-999,Não codificada por logradouros,Total do município
5,Água Doce,89654-000 a 89659-999,Não codificada por logradouros,Total do município
...,...,...,...,...
317,Witmarsum,89157-000 a 89159-999,Não codificada por logradouros,Total do município
318,Xanxerê,89820-000 a 89823-999,Não codificada por logradouros,Total do município
319,Xavantina,89780-000 a 89789-999,Não codificada por logradouros,Total do município
320,Xaxim,89825-000 a 89827-999,Não codificada por logradouros,Total do município


In [5]:
# Resultado no formato JSONL
result = df_noDups.to_json(orient="records", lines=True)

# Printo o resultado no formato JSONL
result

'{"Localidade":"Abdon Batista","Faixa de CEP":"89636-000 a 89637-999","Situa\\u00e7\\u00e3o":"N\\u00e3o codificada por logradouros","Tipo de Faixa":"Total do munic\\u00edpio"}\n{"Localidade":"Abelardo Luz","Faixa de CEP":"89830-000 a 89831-999","Situa\\u00e7\\u00e3o":"N\\u00e3o codificada por logradouros","Tipo de Faixa":"Total do munic\\u00edpio"}\n{"Localidade":"Agrol\\u00e2ndia","Faixa de CEP":"88420-000 a 88429-999","Situa\\u00e7\\u00e3o":"N\\u00e3o codificada por logradouros","Tipo de Faixa":"Total do munic\\u00edpio"}\n{"Localidade":"Agron\\u00f4mica","Faixa de CEP":"89188-000 a 89189-999","Situa\\u00e7\\u00e3o":"N\\u00e3o codificada por logradouros","Tipo de Faixa":"Total do munic\\u00edpio"}\n{"Localidade":"\\u00c1gua Doce","Faixa de CEP":"89654-000 a 89659-999","Situa\\u00e7\\u00e3o":"N\\u00e3o codificada por logradouros","Tipo de Faixa":"Total do munic\\u00edpio"}\n{"Localidade":"\\u00c1guas de Chapec\\u00f3","Faixa de CEP":"89883-000 a 89884-999","Situa\\u00e7\\u00e3o":"N\\u

In [6]:
# Salvando o resultado JSONL na pasta Documents
# Caso apareça a mensagem "FileNotFoundError: [Errno 2] No such file or directory:", 
# colocar um caminho válido no comando abaixo para ser feito o Download do arquivo.
df_noDups.to_json(r'C:\Users\admin\Documents\Export_DataFrame_'+UF_escolha+'.json', orient='records', lines=True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\admin\\Documents\\Export_DataFrame_SC.json'